Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [22]:
batch_size = 128
hidden_size = 1024
l2_weight = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
    
  final_weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  final_biases = tf.Variable(tf.zeros([num_labels]))
    
  def pf(dataset):
    hidden_pre_relu = tf.matmul(dataset, hidden_weights) + hidden_biases    
    return tf.matmul(tf.nn.relu(hidden_pre_relu), final_weights) + final_biases
  
  # Training computation.
  logits = pf(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_weight * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(final_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(pf(tf_valid_dataset))
  test_prediction = tf.nn.softmax(pf(tf_test_dataset))
    
    
    
num_steps = 5001

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 637.864624
Minibatch accuracy: 10.2%
Validation accuracy: 28.5%
Minibatch loss at step 500: 199.581757
Minibatch accuracy: 82.0%
Validation accuracy: 77.8%
Minibatch loss at step 1000: 116.708130
Minibatch accuracy: 78.1%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 68.920891
Minibatch accuracy: 82.8%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 41.430019
Minibatch accuracy: 83.6%
Validation accuracy: 84.1%
Minibatch loss at step 2500: 25.172760
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 15.427338
Minibatch accuracy: 85.2%
Validation accuracy: 86.8%
Minibatch loss at step 3500: 9.504757
Minibatch accuracy: 89.8%
Validation accuracy: 87.3%
Minibatch loss at step 4000: 6.111731
Minibatch accuracy: 85.9%
Validation accuracy: 87.7%
Minibatch loss at step 4500: 3.734743
Minibatch accuracy: 93.0%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 2.563322
Minibatch accuracy: 85.9%
Validati

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [24]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
    
  final_weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  final_biases = tf.Variable(tf.zeros([num_labels]))
    
  def pf(dataset):
    hidden_pre_relu = tf.matmul(dataset, hidden_weights) + hidden_biases    
    return tf.matmul(tf.nn.relu(hidden_pre_relu), final_weights) + final_biases
  
  # Training computation.
  logits = pf(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(pf(tf_valid_dataset))
  test_prediction = tf.nn.softmax(pf(tf_test_dataset))
    
    
    
num_steps = 5001

small_dataset = train_dataset[:batch_size * 20]
small_labels = train_labels[:batch_size * 20]

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_dataset[offset:(offset + batch_size), :]
    batch_labels = small_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 340.480560
Minibatch accuracy: 6.2%
Validation accuracy: 32.7%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 1.649533
Minibatch accuracy: 99.2%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 0.000014
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 0.000001
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 2.517498
Minibatch accuracy: 99.2%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 0.000003
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 3500: 0.000045
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%
Validatio

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [25]:
batch_size = 128
hidden_size = 1024
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
    
  final_weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  final_biases = tf.Variable(tf.zeros([num_labels]))
    
  def pf(dataset, include_dropout=False):
    hidden_pre_relu = tf.matmul(dataset, hidden_weights) + hidden_biases    
    if include_dropout:
        hidden_pre_relu = tf.nn.dropout(hidden_pre_relu, keep_prob)
    return tf.matmul(tf.nn.relu(hidden_pre_relu), final_weights) + final_biases
  
  # Training computation.
  logits = pf(tf_train_dataset, include_dropout=True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(pf(tf_valid_dataset))
  test_prediction = tf.nn.softmax(pf(tf_test_dataset))
    
    
    
num_steps = 5001

small_dataset = train_dataset[:batch_size * 20]
small_labels = train_labels[:batch_size * 20]

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_dataset[offset:(offset + batch_size), :]
    batch_labels = small_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 503.733887
Minibatch accuracy: 6.2%
Validation accuracy: 19.3%
Minibatch loss at step 500: 2.947114
Minibatch accuracy: 96.9%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 0.985855
Minibatch accuracy: 97.7%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.001414
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 12.576542
Minibatch accuracy: 98.4%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 81.3%
Minibatch loss at step 4000: 0.046387
Minibatch accuracy: 99.2%
Validation accuracy: 81.1%
Minibatch loss at step 4500: 0.116645
Minibatch accuracy: 99.2%
Validation accuracy: 81.0%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%
Validation 

In [36]:
batch_size = 128
hidden_size = 1024
keep_prob = 0.5
l2_weight = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_size]))
    
  final_weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  final_biases = tf.Variable(tf.zeros([num_labels]))
    
  def pf(dataset, include_dropout=False):
    hidden_pre_relu = tf.matmul(dataset, hidden_weights) + hidden_biases    
    if include_dropout:
        hidden_pre_relu = tf.nn.dropout(hidden_pre_relu, keep_prob)
    return tf.matmul(tf.nn.relu(hidden_pre_relu), final_weights) + final_biases
  
  # Training computation.
  logits = pf(tf_train_dataset, include_dropout=True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_weight * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(final_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(pf(tf_valid_dataset))
  test_prediction = tf.nn.softmax(pf(tf_test_dataset))
    
    
    
num_steps = 5001

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 802.459717
Minibatch accuracy: 7.8%
Validation accuracy: 33.8%
Minibatch loss at step 500: 204.696869
Minibatch accuracy: 75.8%
Validation accuracy: 78.6%
Minibatch loss at step 1000: 121.942871
Minibatch accuracy: 76.6%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 69.410599
Minibatch accuracy: 78.9%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 41.728531
Minibatch accuracy: 80.5%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 25.347683
Minibatch accuracy: 78.9%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 15.486208
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Minibatch loss at step 3500: 9.538102
Minibatch accuracy: 86.7%
Validation accuracy: 86.2%
Minibatch loss at step 4000: 6.155249
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 4500: 3.828225
Minibatch accuracy: 89.1%
Validation accuracy: 86.8%
Minibatch loss at step 5000: 2.683182
Minibatch accuracy: 81.2%
Validatio

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [56]:
batch_size = 128
hidden_size = 1024
keep_prob = 0.95
l2_weight = 0.0005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  h1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size], stddev=0.05))
  h1_biases = tf.Variable(tf.zeros([hidden_size]))
    
  h2_weights = tf.Variable(
    tf.truncated_normal([hidden_size, hidden_size], stddev=0.05))
  h2_biases = tf.Variable(tf.zeros([hidden_size]))
    
  h3_weights = tf.Variable(
    tf.truncated_normal([hidden_size, hidden_size], stddev=0.05))
  h3_biases = tf.Variable(tf.zeros([hidden_size]))
    
  h4_weights = tf.Variable(
    tf.truncated_normal([hidden_size, hidden_size], stddev=0.05))
  h4_biases = tf.Variable(tf.zeros([hidden_size]))
    
  final_weights = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels], stddev=0.05))
  final_biases = tf.Variable(tf.zeros([num_labels]))
    
  def pf(dataset, include_dropout=False):
    h1 = tf.matmul(dataset, h1_weights) + h1_biases    
    if include_dropout:
        h1 = tf.nn.dropout(h1, keep_prob)
        
    h2 = tf.matmul(tf.nn.relu(h1), h2_weights) + h2_biases    
    if include_dropout:
        h2 = tf.nn.dropout(h2, keep_prob)
        
    h3 = tf.matmul(tf.nn.relu(h2), h3_weights) + h3_biases    
    if include_dropout:
        h3 = tf.nn.dropout(h3, keep_prob)
        
    h4 = tf.matmul(tf.nn.relu(h3), h4_weights) + h4_biases    
    if include_dropout:
        h4 = tf.nn.dropout(h4, keep_prob)
        
    return tf.matmul(tf.nn.relu(h4), final_weights) + final_biases
  
  # Training computation.
  logits = pf(tf_train_dataset, include_dropout=False)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_weight * (
    tf.nn.l2_loss(h1_weights) + tf.nn.l2_loss(h2_weights) + tf.nn.l2_loss(h3_weights) + tf.nn.l2_loss(h4_weights) + tf.nn.l2_loss(final_weights))
 
  # Step Size
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.25, global_step, decay_steps=500, decay_rate=0.96)
    
  # Optimizer.
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(pf(tf_valid_dataset))
  test_prediction = tf.nn.softmax(pf(tf_test_dataset))
    
    
    
num_steps = 95001

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.327099
Minibatch accuracy: 8.6%
Validation accuracy: 27.1%
Minibatch loss at step 500: 1.846367
Minibatch accuracy: 88.3%
Validation accuracy: 85.3%
Minibatch loss at step 1000: 1.403573
Minibatch accuracy: 89.8%
Validation accuracy: 86.8%
Minibatch loss at step 1500: 1.138826
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 2000: 0.876676
Minibatch accuracy: 89.1%
Validation accuracy: 88.5%
Minibatch loss at step 2500: 0.747083
Minibatch accuracy: 91.4%
Validation accuracy: 87.8%
Minibatch loss at step 3000: 0.667563
Minibatch accuracy: 92.2%
Validation accuracy: 88.6%
Minibatch loss at step 3500: 0.598622
Minibatch accuracy: 90.6%
Validation accuracy: 88.9%
Minibatch loss at step 4000: 0.643452
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 4500: 0.460672
Minibatch accuracy: 93.0%
Validation accuracy: 89.2%
Minibatch loss at step 5000: 0.603802
Minibatch accuracy: 85.9%
Validation accuracy